<a href="https://colab.research.google.com/github/tb000330/Research_Assistant/blob/main/Climate%20finance/%EB%B3%80%EC%88%98%EC%84%A0%EC%A0%95(XGB_pannel_df).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
from tqdm import tqdm

import seaborn as sns
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
plt.rc('font', family='NanumGothic')
plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import TimeSeriesSplit 
from matplotlib.patches import Patch 

In [ ]:
panel = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_df.csv')
panel = panel.iloc[:,1:].reset_index(drop=True)
panel.tail(3)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드대분류,코드세분류,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)
111903,A950180,SNK,7.0,2021,0.694562,0.101859,-0.592702,2.830798,9.826040,1.322351,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)


# #1. 전처리 & 파생변수

In [ ]:
panel = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_df.csv')
panel = panel.iloc[:,1:].reset_index(drop=True)

# return 변수 float
panel['return'] = panel['return'].astype(str)
panel['return'] = [re.sub(",", "", row) for row in panel['return']]
panel['return'] = panel['return'].astype(float)

# astype 또는 to_datetime로 변환
# panel['year'] = panel['year'].astype(str)
panel['최초상장일자'] = panel['최초상장일자'].astype(str)
panel['상장일자'] = panel['상장일자'].astype(str)
panel['최초상장일자'] = [re.sub(".0$", "", row) for row in panel['최초상장일자']]
panel['상장일자'] = [re.sub(".0$", "", row) for row in panel['상장일자']]
panel['최초상장일자'] = pd.to_datetime(panel['최초상장일자'])
panel['상장일자'] = pd.to_datetime(panel['상장일자'])

panel['unlist_year'] = panel['unlist_date'].str.slice(stop=4)
panel['unlist_date'] = pd.to_datetime(panel['unlist_date'])
panel['unlist_reason'] = panel['unlist_reason'].astype(str)

# 금융회사 제외
panel = panel[panel['대분류']!='금융 및 보험업']
panel = panel.drop(['대분류', '세분류', '세세분류','소분류', '중분류', '코드세분류', '코드세세분류', '코드소분류', '코드중분류', '주거래은행'], axis=1)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## (1) 재상장 여부 파생변수

In [ ]:
## 재상장 여부 파생변수
relist_list=[]

for f,s in zip(panel['최초상장일자'], panel['상장일자']):
  if f != s:
    relist_list.append(True)
  else:
    relist_list.append(False)

panel['relist'] = relist_list

In [ ]:
panel[panel['최초상장일자']!=panel['상장일자']][['최초상장일자','상장일자','relist']]

,최초상장일자,상장일자,relist
224,2002-10-25,2008-11-28,True
225,2002-10-25,2008-11-28,True
226,2002-10-25,2008-11-28,True
227,2002-10-25,2008-11-28,True
228,2002-10-25,2008-11-28,True
...,...,...,...
111387,NaT,NaT,True
111388,NaT,NaT,True
111389,NaT,NaT,True
111390,NaT,NaT,True


## (2) 상장폐지 사유 전처리 & 카테고리화

In [ ]:
panel['unlist_reason_ctg'] = panel['unlist_reason']

## 상장폐지신청 dict로 class화 
ctg_dict = {}
ctg_list = ['(상장폐지 ?신청)|(^신청)','(신청)|(요청)|(자진)',"(상장$)|(^시장)|(^증권)",'(편입)|(흡수)',"지주회사", "영업의 양도", "기타", "계약이전",
            '해산', "부도", '전액', '자본', "(^감사(으로)? ?의견( 의견)? ?거절)|(범위 ?제한)|(감사의견 ?부적정)|(감사의견거절)", "경영", 
            "(시가 ?총액)|(시총)","주식","보통주", "정리", "영업", "유예", "정지", "제출", "부진", "액면가", "불성실", "화의 불인가", "회생"]

for ctg in ctg_list:
  r = re.compile(ctg) 
  v = list(set([row for row in panel['unlist_reason_ctg'] if r.search(row) is not None]))
  k = min(v, key=len)
  ctg_dict[k] = v
  panel['unlist_reason_ctg'] = panel['unlist_reason_ctg'].replace(v, '0')

# keys 이름 수정
# ctg_dict["자본잠식률 50% 이상"] = ctg_dict["2회연속 자본잠식률 50%이상"]
# ctg_dict["불성실공시법인지정"] = ctg_dict["최근 2년간 3회이상 불성실공시법인으로 지정"]

# del ctg_dict["2회연속 자본잠식률 50%이상"], ctg_dict["최근 2년간 3회이상 불성실공시법인으로 지정"]

In [ ]:
ctg_dict.keys()

dict_keys(['신청에 의한 상장폐지', '자진등록취소', '증권거래소 상장', '피흡수합병', '지주회사(최대주주등)의 완전자회사화 등', '주된영업의 양도', '기타 등록취소', '계약이전', '해산 사유 발생', '부도', '자본전액잠식', '2회연속 자본잠식률 50%이상', '감사의견거절', '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당', '시가총액 미달', '주식양도 제한', '보통주 주가수준미달', '회사정리절차 폐지 결정', '영업활동정지 6월 계속', '상장폐지유예기간종료', '당좌거래정지', '공시서류 미제출(사업보고서) 후 10일이내 미제출', '2분기 연속 거래실적부진사유 미해소', '액면가액일정비율 미달', '최근 2년간 3회이상 불성실공시법인으로 지정', '화의 불인가', '회생절차폐지결정사유 및 당해 코스닥시장상장법인의 계속기업 가능성 유무를 고려한 결과 상장폐지기준에 해당한다고 결정'])

In [ ]:
## dict로 replace
panel['unlist_reason_ctg'] = panel['unlist_reason']

# values에 해당되면 key로 치환
key_list = []

for reason in panel['unlist_reason_ctg']:
  key = [k for k, v in ctg_dict.items() if reason in v]
  if len(key)==0:
    key_list.extend(reason)
  else:
    key_list.extend(key)

panel['unlist_reason_ctg'] = key_list
panel[panel['unlist_reason_ctg']!=panel['unlist_reason']][['unlist_reason_ctg', 'unlist_reason']]

,unlist_reason_ctg,unlist_reason
76864,자본전액잠식,자본전액잠식 2년 계속
76865,자본전액잠식,자본전액잠식 2년 계속
76866,자본전액잠식,자본전액잠식 2년 계속
76867,자본전액잠식,자본전액잠식 2년 계속
76868,자본전액잠식,자본전액잠식 2년 계속
...,...,...
111899,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111900,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111901,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111902,신청에 의한 상장폐지,상장폐지 신청('22.04.14)


## (3) 재무부실기업 상장폐지사유 파생변수

In [ ]:
## 재무적 이유로 상장폐지 하지 않은 사유 필터
panel['fnc_rsn_unlst'] = panel['unlist_reason_ctg']

panel['fnc_rsn_unlst'] = panel['fnc_rsn_unlst'].replace(['해산 사유 발생', '영업활동정지 6월 계속', '자본전액잠식', '회사정리절차 폐지 결정', 
                                                         '감사의견거절', '주식양도 제한', '상장폐지유예기간종료', '부도', 
                                                         '보고서 또는 서류 미제출', '자본잠식률 50% 이상', '시가총액 미달', '보통주 주가수준미달', 
                                                         '당좌거래정지', '액면가액일정비율 미달', '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
                                                         '화의 불인가', '불성실공시법인지정', '2분기 연속 거래실적부진사유 미해소',
                                                         '회생절차폐지결정사유 및 당해 코스닥시장상장법인의 계속기업 가능성 유무를 고려한 결과 상장폐지기준에 해당한다고 결정'],"1")
panel['fnc_rsn_unlst'] = panel['fnc_rsn_unlst'].replace(['피흡수합병','자진등록취소','신청에 의한 상장폐지', '지주회사(최대주주등)의 완전자회사화 등',
                                                          '코스닥시장 상장', '기타 등록취소', '주된영업의 양도','계약이전'],"0")
panel[['unlist_reason_ctg', 'fnc_rsn_unlst']] # {상장폐지X, 그외사유 상장폐지: 0}, {재무사유 상장폐지: 1}

,unlist_reason_ctg,fnc_rsn_unlst
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
111899,신청에 의한 상장폐지,0
111900,신청에 의한 상장폐지,0
111901,신청에 의한 상장폐지,0
111902,신청에 의한 상장폐지,0


## (4) 상장폐지 날짜 정보로 라벨링한 파생변수

In [ ]:
panel['unlist_date'] = panel['unlist_date'].astype(str)
panel['unlist_date'] = panel['unlist_date'].str.slice(stop=4)

In [ ]:
panel['Accounting_Closing'].unique()

array([12.,  6.,  9.,  3.,  5.,  8.,  4.,  7., 11., 10.,  2.,  1.])

## (5) 결측치 제거 

In [ ]:
## 결측값 drop
panel = panel.dropna(subset = panel.iloc[:,4:30].columns,axis = 0).reset_index(drop=True)
panel.apply(lambda x: pd.isna(x).sum()) 

code                                     0
cor_name                                 0
Accounting_Closing                       0
year                                     0
cash/assets                              0
liabilities/assets                       0
liabilities-cash/assets                  0
sales/cur_liabilities                    0
cur_assets/cur_liabilities               0
retained_earnings/cur_liabilities        0
non_cur_lia/non_cur_lia+worth            0
interest/cash_flow                       0
interest/ebitda                          0
inven/sales                              0
sales/net_income                         0
gross_margin/sales                       0
sales/retained_earnings                  0
change_in_roa                            0
change_in_AR                             0
change_in_TP                             0
assets                                   0
sales                                    0
roa                                      0
sales_grow 

In [ ]:
panel.shape

(36084, 39)

# #3. 변수 선정 (시계열 고려하지 않은 XGB)

In [ ]:
def xgb_smltn(X, y):

  X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=1)

  # 객체 생성
  model = XGBClassifier(n_estimators=500, early_stopping_rounds=1, max_depth=3)
  xgb_model = model.fit(X_train, y_train, early_stopping_rounds=100,eval_set=[(X_test, y_test)])

  # 예측하기
  y_pre = xgb_model.predict(X_test)
  y_pred_probs = xgb_model.predict_proba(X_test)[:,1]

  # 특성 중요도 시각화
  # fig, ax = plt.subplots(figsize=(10,12))
  # plot_importance(xgb_model, ax=ax)

  X_test['y_real'] = y_test
  X_test['y_pred'] = y_pre

  print("정확도:",accuracy_score(y_pre, y_test))

  return X_test

## (1) 2차: 재무부실 상장폐지 기업 이진분류

In [ ]:
X2 = panel.iloc[:,4:30] #산업분류코드 제외
# X['relist'] = panel['relist']
y = panel['fnc_rsn_unlst']

In [ ]:
panel[['cor_name','unlist_date', 'fnc_rsn_unlst']]

,cor_name,unlist_date,fnc_rsn_unlst
0,삼성전자,NaT,0
1,삼성전자,NaT,0
2,삼성전자,NaT,0
3,삼성전자,NaT,0
4,삼성전자,NaT,0
...,...,...,...
36079,중국고섬,2013-10-04,1
36080,중국고섬,2013-10-04,1
36081,SNK,2022-05-18,0
36082,SNK,2022-05-18,0


In [ ]:
result2 = xgb_smltn(X2, y)

[0]	validation_0-merror:0.160177
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.15796
[2]	validation_0-merror:0.157822
[3]	validation_0-merror:0.157545
[4]	validation_0-merror:0.157545
[5]	validation_0-merror:0.155743
[6]	validation_0-merror:0.156159
[7]	validation_0-merror:0.157129
[8]	validation_0-merror:0.155189
[9]	validation_0-merror:0.155051
[10]	validation_0-merror:0.154773
[11]	validation_0-merror:0.155051
[12]	validation_0-merror:0.154912
[13]	validation_0-merror:0.154773
[14]	validation_0-merror:0.154358
[15]	validation_0-merror:0.154635
[16]	validation_0-merror:0.154773
[17]	validation_0-merror:0.154773
[18]	validation_0-merror:0.155466
[19]	validation_0-merror:0.154773
[20]	validation_0-merror:0.154635
[21]	validation_0-merror:0.154912
[22]	validation_0-merror:0.154081
[23]	validation_0-merror:0.154358
[24]	validation_0-merror:0.154496
[25]	validation_0-merror:0.154358
[26]	validation_0-merror:0.154635
[27]	validation_0-merror:

In [ ]:
print("정확도:",accuracy_score(result2['y_pred'], result2['y_real']))

정확도: 0.8517389497020923


In [ ]:
print(result2.groupby(['y_real','y_pred'])['y_pred'].count())
print(" - 상장폐지를 상장유지로 예측: 248개","\n","- 상장유지를 상장폐지로 예측: 719개")

y_real                       y_pred                     
0                            0                              5892
                             1                               190
1                            0                               759
                             1                               254
2회연속 자본잠식률 50%이상             0                                40
                             1                                15
공시서류 미제출(사업보고서) 후 10일이내 미제출  0                                26
                             1                                 7
                             공시서류 미제출(사업보고서) 후 10일이내 미제출       1
증권거래소 상장                     0                                28
                             1                                 2
최근 2년간 3회이상 불성실공시법인으로 지정     0                                 3
Name: y_pred, dtype: int64
 - 상장폐지를 상장유지로 예측: 248개 
 - 상장유지를 상장폐지로 예측: 719개


## (2) 3차: 상장폐지 후 마지막 결산재무정보에 라벨링

In [ ]:
## 상장폐지 후 마지막 재무정보에 라벨링
panel['unlist_year'] = '0' 
unlist_cor = list(panel[panel['unlist_date'].notnull()]['cor_name'].unique())
fin_year_list=[]

for cor in unlist_cor:
  fin_year = max(list(panel[panel['cor_name']==cor]['year']))
  fin_idx = panel[(panel['cor_name']==cor)&(panel['year']==fin_year)].index
  panel['unlist_year'][fin_idx] = '1'

## 재무부실사유 상장폐지 아닌 기업 == 0
panel['fnc_rsn_unlst_year'] = panel['fnc_rsn_unlst']
panel['fnc_rsn_unlst_year']  = np.where((panel['fnc_rsn_unlst']=="1")&(panel['unlist_year']=="1"), "1", "0")

panel[panel['unlist_date'].notnull()][['cor_name', 'year','fnc_rsn_unlst','unlist_year','fnc_rsn_unlst_year']]

<ipython-input-19-db21d295a428>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel['unlist_year'][fin_idx] = '1'


,cor_name,year,fnc_rsn_unlst,unlist_year,fnc_rsn_unlst_year
28969,두산상사,1992,1,0,0
28970,두산상사,1993,1,0,0
28971,두산상사,1994,1,0,0
28972,두산상사,1995,1,0,0
28973,두산상사,1996,1,0,0
...,...,...,...,...,...
36079,중국고섬,2011,1,0,0
36080,중국고섬,2012,1,1,1
36081,SNK,2019,0,0,0
36082,SNK,2020,0,0,0


In [ ]:
X3 = panel.loc[:,'cash/assets':'return']
y = panel['fnc_rsn_unlst_year'] 

In [ ]:
result3 = xgb_smltn(X3, y)

[0]	validation_0-error:0.013856
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.013163
[2]	validation_0-error:0.013302
[3]	validation_0-error:0.013302
[4]	validation_0-error:0.013163
[5]	validation_0-error:0.013163
[6]	validation_0-error:0.013163
[7]	validation_0-error:0.013025
[8]	validation_0-error:0.012886
[9]	validation_0-error:0.013163
[10]	validation_0-error:0.013163
[11]	validation_0-error:0.013163
[12]	validation_0-error:0.013163
[13]	validation_0-error:0.013163
[14]	validation_0-error:0.013163
[15]	validation_0-error:0.012886
[16]	validation_0-error:0.012748
[17]	validation_0-error:0.012748
[18]	validation_0-error:0.012886
[19]	validation_0-error:0.013025
[20]	validation_0-error:0.012886
[21]	validation_0-error:0.012886
[22]	validation_0-error:0.012886
[23]	validation_0-error:0.013025
[24]	validation_0-error:0.013025
[25]	validation_0-error:0.013163
[26]	validation_0-error:0.013025
[27]	validation_0-error:0.013302
[28]	validation_0-e

In [ ]:
print("정확도:",accuracy_score(result3['y_pred'], result3['y_real']))

정확도: 0.987252320908965


In [ ]:
print(result3.groupby(['y_real','y_pred'])['y_pred'].count())
print(" - 상장폐지를 상장유지로 예측: 10개","\n","- 상장유지를 상장폐지로 예측: 85개")

y_real  y_pred
0       0         7109
        1            5
1       0           87
        1           16
Name: y_pred, dtype: int64
 - 상장폐지를 상장유지로 예측: 10개 
 - 상장유지를 상장폐지로 예측: 85개


In [ ]:
#예측오류 값 분석
panel_Xtest = pd.concat([panel.loc[result3.index,:],result3['y_pred']], axis=1)
panel_Xtest[panel_Xtest['fnc_rsn_unlst_year']!=panel_Xtest['y_pred']].groupby('unlist_reason_ctg')['unlist_reason_ctg'].count().sort_values(ascending=False)

unlist_reason_ctg
감사의견거절                                        36
해산 사유 발생                                      14
기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당     9
부도                                             8
자본전액잠식                                         8
회사정리절차 폐지 결정                                   6
상장폐지유예기간종료                                     3
기타 등록취소                                        2
증권거래소 상장                                       2
2분기 연속 거래실적부진사유 미해소                            1
영업활동정지 6월 계속                                   1
주식양도 제한                                        1
화의 불인가                                         1
Name: unlist_reason_ctg, dtype: int64

## (3) 4차: 라벨링 반영 - 기업코드

In [ ]:
## 기업코드 레이블 인코딩
panel['code_label'] = [int(re.sub("^A", "", row) )for row in panel['code']]

In [ ]:
X4 = panel.loc[:,'cash/assets':'return']
X4['code_label'] = panel['code_label']

y = panel['fnc_rsn_unlst_year'] 

In [ ]:
## 딕셔너리화 {기업코드_int:기업코드}
code_book = pd.concat([X4[['code_label']],panel['code']],axis=1)

def df_to_dict(df, keys_col, values_col):
      
  trans = df.loc[:,[keys_col, values_col]].transpose()
  trans.columns = trans.iloc[0]
  trans = trans.iloc[1].to_dict()

  for k in list(trans.keys()):
    if pd.isnull(trans[k])==True:
      del trans[k]

  return trans

code_dict = df_to_dict(code_book, 'code_label','code')
print(code_dict)
code_book

{5930: 'A005930', 660: 'A000660', 207940: 'A207940', 6400: 'A006400', 51910: 'A051910', 5380: 'A005380', 270: 'A000270', 68270: 'A068270', 5490: 'A005490', 28260: 'A028260', 35720: 'A035720', 12330: 'A012330', 3670: 'A003670', 96770: 'A096770', 34730: 'A034730', 66570: 'A066570', 33780: 'A033780', 3550: 'A003550', 10130: 'A010130', 15760: 'A015760', 17670: 'A017670', 91990: 'A091990', 247540: 'A247540', 9150: 'A009150', 34020: 'A034020', 10950: 'A010950', 11200: 'A011200', 18260: 'A018260', 329180: 'A329180', 30200: 'A030200', 9830: 'A009830', 259960: 'A259960', 3490: 'A003490', 51900: 'A051900', 36570: 'A036570', 66970: 'A066970', 11070: 'A011070', 86280: 'A086280', 302440: 'A302440', 90430: 'A090430', 97950: 'A097950', 11170: 'A011170', 9540: 'A009540', 352820: 'A352820', 326030: 'A326030', 32640: 'A032640', 34220: 'A034220', 35250: 'A035250', 28050: 'A028050', 267250: 'A267250', 10140: 'A010140', 47810: 'A047810', 720: 'A000720', 78930: 'A078930', 11780: 'A011780', 18880: 'A018880',

,code_label,code
0,5930,A005930
1,5930,A005930
2,5930,A005930
3,5930,A005930
4,5930,A005930
...,...,...
36079,950070,A950070
36080,950070,A950070
36081,950180,A950180
36082,950180,A950180


In [ ]:
result4 = xgb_smltn(X4, y)

[0]	validation_0-error:0.013856
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.013163
[2]	validation_0-error:0.013302
[3]	validation_0-error:0.013302
[4]	validation_0-error:0.013163
[5]	validation_0-error:0.013163
[6]	validation_0-error:0.013163
[7]	validation_0-error:0.013025
[8]	validation_0-error:0.012886
[9]	validation_0-error:0.013163
[10]	validation_0-error:0.013163
[11]	validation_0-error:0.013163
[12]	validation_0-error:0.013163
[13]	validation_0-error:0.013163
[14]	validation_0-error:0.013163
[15]	validation_0-error:0.012886
[16]	validation_0-error:0.012748
[17]	validation_0-error:0.012748
[18]	validation_0-error:0.012886
[19]	validation_0-error:0.013025
[20]	validation_0-error:0.012886
[21]	validation_0-error:0.012886
[22]	validation_0-error:0.012886
[23]	validation_0-error:0.013025
[24]	validation_0-error:0.013025
[25]	validation_0-error:0.013163
[26]	validation_0-error:0.013025
[27]	validation_0-error:0.013302
[28]	validation_0-e

In [ ]:
print("정확도:",accuracy_score(result4['y_pred'], result4['y_real']))

정확도: 0.987252320908965


In [ ]:
print(result4.groupby(['y_real','y_pred'])['y_pred'].count())
print(" - 상장폐지를 상장유지로 예측: 10개","\n","- 상장유지를 상장폐지로 예측: 85개")

y_real  y_pred
0       0         7109
        1            5
1       0           87
        1           16
Name: y_pred, dtype: int64
 - 상장폐지를 상장유지로 예측: 10개 
 - 상장유지를 상장폐지로 예측: 85개


## (4) 5차: 라벨링 반영 - 연도

In [ ]:
X5 = panel.loc[:,'cash/assets':'return']
X5['year'] = panel['year']

y = panel['fnc_rsn_unlst_year'] 

In [ ]:
panel[['year']]

,year
0,1992
1,1993
2,1994
3,1995
4,1996
...,...
36079,2011
36080,2012
36081,2019
36082,2020


In [ ]:
result5 = xgb_smltn(X5, y)

[0]	validation_0-error:0.013856
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.013163
[2]	validation_0-error:0.013302
[3]	validation_0-error:0.013302
[4]	validation_0-error:0.013163
[5]	validation_0-error:0.013163
[6]	validation_0-error:0.013163
[7]	validation_0-error:0.013163
[8]	validation_0-error:0.013163
[9]	validation_0-error:0.013302
[10]	validation_0-error:0.013302
[11]	validation_0-error:0.013025
[12]	validation_0-error:0.012748
[13]	validation_0-error:0.013025
[14]	validation_0-error:0.013163
[15]	validation_0-error:0.013163
[16]	validation_0-error:0.013025
[17]	validation_0-error:0.013163
[18]	validation_0-error:0.012886
[19]	validation_0-error:0.013163
[20]	validation_0-error:0.013025
[21]	validation_0-error:0.012886
[22]	validation_0-error:0.012886
[23]	validation_0-error:0.012886
[24]	validation_0-error:0.012886
[25]	validation_0-error:0.013163
[26]	validation_0-error:0.013025
[27]	validation_0-error:0.012886
[28]	validation_0-e

In [ ]:
print("정확도:",accuracy_score(result5['y_pred'], result5['y_real']))

정확도: 0.987252320908965


In [ ]:
print(result5.groupby(['y_real','y_pred'])['y_pred'].count())
print(" - 상장폐지를 상장유지로 예측: 10개","\n","- 상장유지를 상장폐지로 예측: 85개")

y_real  y_pred
0       0         7110
        1            4
1       0           88
        1           15
Name: y_pred, dtype: int64
 - 상장폐지를 상장유지로 예측: 10개 
 - 상장유지를 상장폐지로 예측: 85개


## (5) 6차: 라벨링 반영 - 기업코드, 연도

In [ ]:
X6 = panel.loc[:,'cash/assets':'return']
X6['year'] = panel['year']
X6['code_label'] = panel['code_label']

y = panel['fnc_rsn_unlst_year'] 

In [ ]:
result6 = xgb_smltn(X6, y)

[0]	validation_0-error:0.013856
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.013163
[2]	validation_0-error:0.013302
[3]	validation_0-error:0.013302
[4]	validation_0-error:0.013163
[5]	validation_0-error:0.013163
[6]	validation_0-error:0.013163
[7]	validation_0-error:0.013163
[8]	validation_0-error:0.013163
[9]	validation_0-error:0.013302
[10]	validation_0-error:0.013302
[11]	validation_0-error:0.013025
[12]	validation_0-error:0.012748
[13]	validation_0-error:0.013025
[14]	validation_0-error:0.013163
[15]	validation_0-error:0.013163
[16]	validation_0-error:0.013025
[17]	validation_0-error:0.013163
[18]	validation_0-error:0.012886
[19]	validation_0-error:0.013163
[20]	validation_0-error:0.013025
[21]	validation_0-error:0.012886
[22]	validation_0-error:0.012886
[23]	validation_0-error:0.012886
[24]	validation_0-error:0.012886
[25]	validation_0-error:0.013163
[26]	validation_0-error:0.013025
[27]	validation_0-error:0.012886
[28]	validation_0-e

In [ ]:
print("정확도:",accuracy_score(result6['y_pred'], result6['y_real']))

정확도: 0.987252320908965


In [ ]:
print(result6.groupby(['y_real','y_pred'])['y_pred'].count())
print(" - 상장폐지를 상장유지로 예측: 8개","\n","- 상장유지를 상장폐지로 예측: 85개")

y_real  y_pred
0       0         7110
        1            4
1       0           88
        1           15
Name: y_pred, dtype: int64
 - 상장폐지를 상장유지로 예측: 8개 
 - 상장유지를 상장폐지로 예측: 85개


## (6) 정확도 기준(난중에 손보기)

In [ ]:
result6 = result6.replace({'code_label':code_dict})
result6[result6['y_pred']=='1'][['code_label','y_pred']].drop_duplicates()['code_label'].values

array(['A047370', 'A074130', 'A007480', 'A017210', 'A089240', 'A039040',
       'A035400', 'A036020', 'A031800', 'A033190', 'A105070', 'A067950',
       'A034970', 'A019640', 'A053170', 'A040020', 'A058690', 'A043680',
       'A013240'], dtype=object)

In [ ]:
panel[panel['fnc_rsn_unlst_year']=='1'][['code','fnc_rsn_unlst_year']].drop_duplicates()['code'].values

array(['A000090', 'A000160', 'A000170', 'A000200', 'A000280', 'A000310',
       'A000330', 'A000360', 'A000420', 'A000450', 'A000510', 'A000730',
       'A000840', 'A000900', 'A000930', 'A001150', 'A001160', 'A001300',
       'A001350', 'A001370', 'A001400', 'A001580', 'A001600', 'A001610',
       'A001670', 'A001690', 'A001700', 'A001830', 'A001920', 'A001930',
       'A001980', 'A002010', 'A002050', 'A002080', 'A002130', 'A002180',
       'A002270', 'A002300', 'A002340', 'A002440', 'A002540', 'A002610',
       'A002670', 'A002740', 'A002850', 'A002910', 'A002930', 'A002940',
       'A003020', 'A003040', 'A003110', 'A003190', 'A003290', 'A003430',
       'A003500', 'A003590', 'A003640', 'A003840', 'A003900', 'A003930',
       'A003940', 'A003970', 'A003990', 'A004010', 'A004110', 'A004130',
       'A004190', 'A004200', 'A004210', 'A004230', 'A004260', 'A004280',
       'A004290', 'A004320', 'A004390', 'A004460', 'A004530', 'A004600',
       'A004620', 'A004730', 'A004820', 'A005000', 

# #4. 시계열 변수

## (1) 1차: window method

In [ ]:
panel['year'] = panel['year'].astype(int)
panel_time = panel.sort_values(by='year').set_index('year')
panel_time = panel_time[['cash/assets', 'liabilities/assets', 'liabilities-cash/assets', 'sales/cur_liabilities', 'cur_assets/cur_liabilities', 'retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth','interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings','change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow', 'closing_price','revised_price', 'market_capital', 'return','code_label', 'fnc_rsn_unlst_year']]
panel_time.head()

,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,non_cur_lia/non_cur_lia+worth,interest/cash_flow,interest/ebitda,inven/sales,...,roa,sales_grow,net_income,net_income_grow,closing_price,revised_price,market_capital,return,code_label,fnc_rsn_unlst_year
year,,,,,,,,,,,,,,,,,,,,,
1990,0.049452,0.445126,0.395674,2.016473,1.720471,0.297284,0.196518,0.000000,0.00000,0.376660,...,1.80,-4.15,760674.0,-3.13,11673.0,2270.0,18677.0,-31.71,4920,0
1990,0.043560,0.159283,0.115723,6.025217,2.502392,1.698017,0.054765,0.000000,0.00000,0.137048,...,3.49,1.27,916288.0,-11.72,12489.0,677.0,11240.0,-25.58,1620,0
1990,0.025339,0.550153,0.524813,3.646023,1.155612,1.067915,0.377004,0.000000,0.00000,0.131562,...,1.90,-3.14,2089582.0,-55.19,49377.0,2560.0,53327.0,0.82,50,0
1990,0.075825,0.778589,0.702763,1.136746,0.953148,0.291547,0.651035,0.000517,0.00027,0.153906,...,1.89,26.88,12157453.0,-65.27,27816.0,65906.0,250976.0,-36.64,1520,0
1990,0.027545,0.392510,0.364965,5.164983,3.116677,0.855841,0.257223,0.000000,0.00000,0.156199,...,8.55,3.59,21500406.0,29.76,22741.0,17056.0,272898.0,-35.92,10130,0


In [ ]:
def xgb_model(X_train, y_train, X_val, y_val):
  model = XGBClassifier(n_estimators=500, early_stopping_rounds=1, max_depth=3)
  xgb_model = model.fit(X_train, y_train, early_stopping_rounds=100, eval_set=[(X_val, y_val)])
  return xgb_model

In [ ]:
def xgb_prediction_1(df, test_start_year):

  result = pd.DataFrame(columns=df.loc[:,'cash/assets':'code_label'].columns)
  real_y_lst = []
  pred_y_lst = []
  pred_score_lst = [] 
  pred_vlus_lst = [] 

  df_train = df[df.index < int(test_start_year)] # 학습데이터
  window = len(set(df_train.index)) # 윈도우 사이즈 설정 (2002~2015)

  df_test = df[df.index >= int(test_start_year)] #검증데이터

  start_index = 1990

  # fit the model and make a forecast
  for year in list(set(df_test.index)): #연도 별

      end_index = start_index + window 
      new_data = df.loc[start_index : end_index] # df에서 end_index+1

      x_train = new_data.loc[:(end_index-1),'cash/assets':'code_label']
      x_val = new_data.loc[(end_index-1),'cash/assets':'code_label'] #test set연도 이전연도

      y_train = new_data.loc[:(end_index-1),"fnc_rsn_unlst_year"]
      y_val = new_data.loc[(end_index-1),"fnc_rsn_unlst_year"] #test set연도의 이전연도
      # x_train, x_val, y_train, y_val = train_test_split(new_data.loc[:,'cash/assets':'return'], new_data["unlist_year"] ,test_size = 0.25, shuffle = False)

      # gets the trained xgb model with given data
      model = xgb_model(x_train, y_train, x_val, y_val) 

      x_new = df.loc[end_index] #test set
      x_new = pd.DataFrame(x_new).loc[:,'cash/assets':'code_label']

      y_pred = model.predict(x_new)
      y_pred_probs = model.predict_proba(x_new)

      result = pd.concat([result,x_new], axis=0)
      pred_y_lst.extend(y_pred)
      pred_score_lst.extend(y_pred_probs[:,0])
      pred_vlus_lst.extend(y_pred_probs[:,1])

      start_index = start_index + 1 # for rolling-window forecasts

      # window = window + 1 # for recursive forecasts
  result['y'] = df.loc[test_start_year:,'fnc_rsn_unlst_year']
  result['pred_y'] = pred_y_lst
  result['pred_y'] = pred_y_lst
  result['pred_score'] = pred_score_lst
  result['pred_vlus'] = pred_vlus_lst

  return result

In [ ]:
result = xgb_prediction_1(panel_time, 2013)

[0]	validation_0-error:0.011565
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.011565
[2]	validation_0-error:0.011565
[3]	validation_0-error:0.011565
[4]	validation_0-error:0.011565
[5]	validation_0-error:0.011565
[6]	validation_0-error:0.011565
[7]	validation_0-error:0.011565
[8]	validation_0-error:0.011565
[9]	validation_0-error:0.011565
[10]	validation_0-error:0.011565
[11]	validation_0-error:0.011565
[12]	validation_0-error:0.011565
[13]	validation_0-error:0.011565
[14]	validation_0-error:0.011565
[15]	validation_0-error:0.011565
[16]	validation_0-error:0.011565
[17]	validation_0-error:0.011565
[18]	validation_0-error:0.011565
[19]	validation_0-error:0.011565
[20]	validation_0-error:0.011565
[21]	validation_0-error:0.011565
[22]	validation_0-error:0.011565
[23]	validation_0-error:0.011565
[24]	validation_0-error:0.011565
[25]	validation_0-error:0.011565
[26]	validation_0-error:0.011565
[27]	validation_0-error:0.011565
[28]	validation_0-e

In [ ]:
print("정확도:",accuracy_score(result['pred_y'], result['y']))
result

정확도: 0.9927960378208014


,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,non_cur_lia/non_cur_lia+worth,interest/cash_flow,interest/ebitda,inven/sales,...,net_income_grow,closing_price,revised_price,market_capital,return,code_label,y,pred_y,pred_score,pred_vlus
2013,0.119170,0.687388,0.568218,3.864107,1.248529,0.377269,0.465247,-0.680142,3.184551,0.068465,...,-5.031114,17197.0,15581.0,262729.0,-41.42,5950,0,0,0.992001,0.007999
2013,0.218641,0.203804,-0.014837,7.213405,2.684249,-0.929985,0.000000,0.092320,0.065090,0.028407,...,-91.650000,1073.0,1073.0,15527.0,-7.35,43580,0,0,0.971918,0.028082
2013,0.014353,0.669185,0.654832,2.673306,0.755377,1.620972,0.619766,0.319616,0.267289,0.079196,...,-1.056630,30211.0,30211.0,19394203.0,14.12,15760,0,0,0.993852,0.006148
2013,0.108204,0.114284,0.006080,5.802681,4.808713,5.059385,0.000137,-0.068951,0.030954,0.001836,...,-0.560000,4961.0,4961.0,65456.0,-30.56,72130,0,0,0.992815,0.007185
2013,0.110328,0.538397,0.428070,2.085081,1.158151,-2.364223,0.066171,1.309002,0.460945,0.088023,...,-1.024751,1451.0,1451.0,28283.0,6.53,43090,0,0,0.979448,0.020552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,0.163028,0.462214,0.299186,3.608521,1.941366,1.337274,0.287564,0.068767,0.038846,0.199738,...,70.840000,45436.0,45436.0,2760343.0,-17.96,81660,0,0,0.593194,0.406806
2021,0.186706,0.573762,0.387056,1.877338,1.303089,-0.056183,0.085881,0.560344,0.465114,0.154371,...,-38.490000,1961.0,1961.0,96853.0,46.97,27580,0,0,0.593194,0.406806
2021,0.112923,0.681238,0.568315,2.800715,1.042487,0.157552,0.327397,0.322997,0.281694,0.100712,...,-1.031895,4386.0,4386.0,175422.0,23.45,6380,0,0,0.593194,0.406806
2021,0.054605,0.170543,0.115938,0.811645,2.656752,2.630772,0.009152,0.011231,0.033757,0.220925,...,-3.098563,4456.0,4456.0,502931.0,9.34,126560,0,0,0.593194,0.406806


In [ ]:
print(result.groupby(['y','pred_y'])['pred_y'].count())
print(" - 상장폐지를 상장유지로 예측: 12개","\n","- 상장유지를 상장폐지로 예측: 100개")

y  pred_y
0  0         15411
   1            12
1  0           100
   1            24
Name: pred_y, dtype: int64
 - 상장폐지를 상장유지로 예측: 12개 
 - 상장유지를 상장폐지로 예측: 100개


## (2) 2차: 시차 반영(전년도 재무정보로만 다음연도 상장폐지예측)

- $X_t \; ←predict- \; X_{t-s} \quad (if \;\, s>1)$

In [ ]:
panel_shift = panel[['year','cash/assets', 'liabilities/assets', 'liabilities-cash/assets', 'sales/cur_liabilities', 'cur_assets/cur_liabilities', 'retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth','interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings','change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow', 'closing_price','revised_price', 'market_capital', 'return','code_label', 'fnc_rsn_unlst_year']]

def cor_shift(df, period):
  shift_result = pd.DataFrame()

  for cor in df['code_label'].unique():
    df_cor = df[df['code_label']==cor]
    df_cor['fnc_rsn_unlst_year'] = df_cor['fnc_rsn_unlst_year'].shift(periods=period)
    shift_result = pd.concat([shift_result, df_cor], axis=0)

  return shift_result

panel_shift = cor_shift(panel_shift, 1)
panel_shift

<ipython-input-68-8217bade82bb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cor['fnc_rsn_unlst_year'] = df_cor['fnc_rsn_unlst_year'].shift(periods=period)


,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,non_cur_lia/non_cur_lia+worth,interest/cash_flow,interest/ebitda,...,roa,sales_grow,net_income,net_income_grow,closing_price,revised_price,market_capital,return,code_label,fnc_rsn_unlst_year
0,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,0.656881,0.001486,0.000561,...,1.22,16.75,7.243830e+07,5.550000,30431.0,298.0,1464099.0,8.17,5930,NaN
1,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,0.649281,0.000000,0.000000,...,1.04,42.31,7.648482e+07,5.590000,39953.0,407.0,2155136.0,77.95,5930,0
2,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,0.501214,0.000000,0.000000,...,11.11,35.44,1.002286e+09,1210.440000,96710.0,1011.0,5974577.0,94.40,5930,0
3,1995,0.054644,0.693142,0.638497,2.801825,1.333899,0.514395,0.494786,0.001220,0.000839,...,15.90,58.71,2.112871e+09,110.810000,130982.0,1654.0,9692485.0,59.74,5930,0
4,1996,0.055147,0.777430,0.722283,2.550588,1.255358,0.424394,0.654395,0.004928,0.003075,...,-0.41,6.47,-7.950894e+07,-1.037631,83895.0,1353.0,8079958.0,-56.81,5930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36079,2011,0.223984,0.212003,-0.011981,1.656090,1.376470,-1.459327,0.000000,-0.000000,-0.000000,...,-14.89,-69.54,-3.713026e+07,-0.619974,4272.0,4272.0,128155.0,-29.41,950070,NaN
36080,2012,0.008260,0.425390,0.417130,0.599048,0.343097,-0.671742,0.000000,-0.000000,-0.000000,...,-4.90,-12.82,-1.544306e+07,-0.584084,4165.0,4165.0,124950.0,0.00,950070,0
36081,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,0.010316,0.000000,0.000000,...,14.64,50.29,3.116920e+07,25.480000,25852.0,25852.0,544482.0,-52.45,950180,NaN
36082,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,0.014456,0.006268,0.005792,...,6.70,-15.74,2.265319e+07,-27.320000,16598.0,16598.0,349583.0,41.56,950180,0


In [ ]:
def xgb_prediction_2(df, test_start_year):

  result = pd.DataFrame(columns=df.loc[:,'cash/assets':'code_label'].columns)
  real_y_lst = []
  pred_y_lst = []
  pred_score_lst = []
  pred_vlus_lst = []

  df_train = df[df.index < int(test_start_year)] # 학습데이터
  window = len(set(df_train.index)) # 윈도우 사이즈 설정 (2002~2015)

  df_test = df[df.index >= int(test_start_year)] #검증데이터

  start_index = 1990

  # fit the model and make a forecast
  for year in list(set(df_test.index)): #연도 별

      end_index = start_index + window 
      new_data = df.loc[start_index : end_index] # df에서 end_index+1

      x_train = new_data.loc[:(end_index-1),'cash/assets':'code_label']
      x_val = new_data.loc[(end_index-1),'cash/assets':'code_label'] #test set연도 이전연도

      y_train = new_data.loc[:(end_index-1),"fnc_rsn_unlst_year"]
      y_val = new_data.loc[(end_index-1),"fnc_rsn_unlst_year"] #test set연도의 이전연도
      # x_train, x_val, y_train, y_val = train_test_split(new_data.loc[:,'cash/assets':'return'], new_data["unlist_year"] ,test_size = 0.25, shuffle = False)

      # gets the trained xgb model with given data
      model = xgb_model(x_train, y_train, x_val, y_val) 

      x_new = df.loc[end_index] #test set
      x_new = pd.DataFrame(x_new).loc[:,'cash/assets':'code_label']

      y_pred = model.predict(x_new)
      y_pred_probs = model.predict_proba(x_new)

      result = pd.concat([result,x_new], axis=0)
      pred_y_lst.extend(y_pred)
      pred_score_lst.extend(y_pred_probs[:,0])
      pred_vlus_lst.extend(y_pred_probs[:,1])

      start_index = start_index + 1 # for rolling-window forecasts

      # window = window + 1 # for recursive forecasts
  result['y'] = df.loc[test_start_year:,'fnc_rsn_unlst_year']
  result['pred_y'] = pred_y_lst
  result['pred_y'] = pred_y_lst
  result['pred_score'] = pred_score_lst
  result['pred_vlus'] = pred_vlus_lst

  return result

In [ ]:
r = re.compile(" [0-9]년")
set([row for row in panel['unlist_reason'] if r.search(row) is not None])

{'감사의견 의견거절 2년 계속',
 '감사의견거절, 자본전액잠식, 2년 연속 자본잠식률 50% 이상',
 '보통주 주식분포요건 미충족(일반주주 주식수 10/100미만) 2년 계속',
 '신고,공시의무를 위반하여 투자유의종목으로 지정된 후 1년이내에 불성실공시법인으로 지정 및 최근 2년간 3회 이상 불성실공시법인으로 지정',
 '자본잠식(자본금의 50/100이상 잠식) 2년 계속',
 '자본전액잠식 2년 계속',
 '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조제1항에 따른 사업보고서 법정제출기한 내 미제출, 최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
 '주식분산기준 미달 2년 연속',
 '최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
 '최근 2년간 3회이상 불성실공시법인으로 지정'}

In [ ]:
len_list=[]

for cor in panel['cor_name'].unique():
   len_list.append(len(panel[panel['cor_name']==cor]))

In [ ]:
pd.DataFrame(len_list).groupby(0)[0].count().sort_values(ascending=False)

0
28    247
10    182
3     154
4     154
8     152
6     149
7     140
5     138
2     129
21    122
9     121
18    114
16    108
1     100
13     99
12     95
11     89
14     83
15     75
19     75
17     74
23     62
22     39
30     38
20     37
24     29
27     20
25     17
26     12
29      3
32      1
34      1
35      1
Name: 0, dtype: int64